# Google Earth Engine - Python API - Vegetation Change Methods - Case Study - Tolima Department, Columbia - 2017 Semester A Growing Season

## Introduction

This workflow implements a vegetation change detection analysis with the Google Earth Engine Python API. The workflow applies these methods to a study area in the Tolimna Department, Columbia, in the 2017 Semester A growing season, from peak green (June) to post-harvest (September).

## Environment Setup

This workflow uses Python 3.8 and requires the following packages:

* *ee*
* *geemap*
* *vegetation_change*

The *ee* and *geemap* packages are available from [Conda-Forge](https://conda-forge.org/). The *vegetation_change* package (from *vegetation_change.py*) provides custom functions that implement Google Earth Engine functionality for the benefit of this analysis. The Conda environment provided with this analysis (contained in *environment.yml*) includes all packages needed from Conda-Forge. The custom script is provided in the same folder as this Jupyter Notebook.

In [1]:
# Import packages
import ee
import geemap as gm
import vegetation_change as vc

The workflow authenticates to Google Earth Engine with an active account and then initializes the Google Earth Engine library (if the authentication succeeds).

In [2]:
# Initialze GEE Python API; authenticate if necessary
try:
    ee.Initialize()
    
except Exception as error:
    ee.Authenticate()
    ee.Initialize()

The workflow defines the user name and public folder for accessing the study area features within and exporting the results of the analysis to Google Earth Engine Assets. The import of the study area features step must use these variables, unless a user has exported them to another location prior to the analysis. The export of the results may omit this path and use a path specific user who executes the workflow.

In [3]:
# Define output folder (GEE username + folder location)
gee_username = "calekochenour"
gee_asset_folder = "vegetation-change" 

## Data Preparation

The workflow imports the study area boundary and study area canals, each as individual `ee.FeatureCollection` object.

In [4]:
# Create FeatureCollections for study area
study_area_boundary = ee.FeatureCollection(
    f'users/{gee_username}/{gee_asset_folder}/drtt_study_area_boundary')

study_area_canals = ee.FeatureCollection(
    f'users/{gee_username}/{gee_asset_folder}/drtt_study_area_canals')

The workflow loads two Landsat 8 images, June 2, 2017 (peak green) and September 6, 2017 (post-harvest) as `ee.Image` objects and and clips each image to the study area boundary. Then the workflow creates an `ee.ImageCollection` object that contains the two images.

In [5]:
# Load and clip imagery for 2017 Semester A growing season
peak_green = ee.Image(
    'LANDSAT/LC08/C01/T1_SR/LC08_008057_20170602').clip(study_area_boundary)

post_harvest = ee.Image(
    'LANDSAT/LC08/C01/T1_SR/LC08_008057_20170906').clip(study_area_boundary)

# Create ImageCollection for Peak Green and Post-Harvest
collection = ee.ImageCollection([peak_green, post_harvest])

## Data Processing

The workflow uses a custom function from the `vegetation_change.py` script, `ndvi_diff_landsat8()` to:

* Mask each image in the `ee.ImageCOllection` for clouds and cloud shadows;
* Compute and add the normalized difference vegetation index (NDVI) band to each image in the `ee.ImageCOllection`;
* Convert the `ee.ImageCOllection` to an `ee.List` object;
* Subtract the peak green NDVI band from the post-harvest NDVI band; and,
* Return the NDVI difference (post-harvest NDVI - peak green NDVI).

In [6]:
# Compute NDVI difference raster for Peak Green to Post-Harvest
ndvi_diff = vc.ndvi_diff_landsat8(collection, 1, 0)

The workflow defines two threshold ranges for NDVI change, -2.0 to -0.5 for the primary change (largest change) and -0.5 to -0.35 for secondary change (large, but less large). The workflow refined these thresholds based on the specific imagery used in this study area, during the time period analyzed. 

In [7]:
# Define NDVI thresholds for classification;
#  indices 0/1 identify min/max for primary class;
#  indices 2/3 identify min/max for secondary class
ndvi_change_thresholds = [-2.0, -0.5, -0.5, -0.35]

The workflow uses a custom function from the `vegetation_change.py` script, `segment_snic()` to:

* Segment the NDVI difference image;
* Classify the image based on the define NDVI thresholds;
* Extract classified features (primary and secondary NDVI difference ranges); and,
* Return the extracted features as `ee.Image` objects stored in a dictionary.

In [8]:
# Segment, classify, and extract features
change_features = vc.segment_snic(
    ndvi_diff, study_area_boundary, ndvi_change_thresholds)

The workflow uses a custom function from the `vegetation_change.py` script, `raster_to_vector()` to:

* Convert the classified extracted features (in `ee.Image` format) to `ee.FeatureCollection` objects; and,
* Return the vectorized version of the extracted features.

In [9]:
# Convert rasters to vectors
change_primary_vector = vc.raster_to_vector(
    change_features.get('primary'), study_area_boundary)

change_secondary_vector = vc.raster_to_vector(
    change_features.get('secondary'), study_area_boundary)

## Data Export

The workflow defines output folder locations, within a valid Google Earth Engine Assets folder and uses a custom function from the `vegetation_change.py` script, `export_vector()` to export the vectorized extracted features to the defined Assets folder. 

A user must change the `gee_username` variable to the user name used to authenticate to to Google Earth Engine at the beginning of this workflow in order for the export to succeed. The user must also change the `gee_asset_folder` to a valid Assets folder within the user account. A user may change the name of the output files, `vegetation_change_primary` and `vegetation_change_secondary`.

The workflow implements a check for the existence of the file path and output file names prior to exporting and skips the export if a file with the specified file name already exists.

In [10]:
# Define output GEE Asset names
change_primary_asset_name = f'users/{gee_username}/{gee_asset_folder}/vegetation_change_primary'
change_secondary_asset_name = f'users/{gee_username}/{gee_asset_folder}/vegetation_change_secondary'

# Check if GEE Asset already exists prior to export; primary change
if (change_primary_asset := ee.FeatureCollection(change_primary_asset_name)):

    # Skip export
    print(
        f"GEE Asset ID '{change_primary_asset_name}' already exists. Skipping export...")

else:
    # Export vectors to GEE Asset
    change_primary_export = vc.export_vector(
        vector=change_primary_vector,
        description='Primary Change',
        output_name=change_primary_asset_name,
        output_method='asset')

# Check if GEE Asset already exists prior to export; secondary change
if (change_secondary_asset := ee.FeatureCollection(change_secondary_asset_name)):

    # Skip export
    print(
        f"GEE Asset ID '{change_secondary_asset_name}' already exists. Skipping export...")

else:
    # Export vectors to GEE Asset
    change_secondary_export = vc.export_vector(
        vector=change_secondary_vector,
        description='Secondary Change',
        output_name=change_secondary_asset_name,
        output_method='asset')

GEE Asset ID 'users/calekochenour/vegetation-change/vegetation_change_primary' already exists. Skipping export...
GEE Asset ID 'users/calekochenour/vegetation-change/vegetation_change_secondary' already exists. Skipping export...


## Data Visualization

The workflow uses a function from the `geemap` package, `Map()`, to create an interactive map for displaying layers in the analysis. 

The workflow uses the a function from the `geemap` package, `addLayer()`, to add all layers from the analysis to the interactive map, to include:

* Study area vector files;
* Pre and post-change imagery (RGB and CIR); 
* NDVI difference image;
* Classified clusters (rasters); 
* Classified clusters (vectors) from the raster to vector conversion; and,
* Classified clusters (vectors) imported from the Google Earth Engine Assets which the workflow exported during the Data Export step. 

The workflow defines the display of each layers with visualization parameters, some with single colors (vector layers, classified/extracted features), and others with pre-defined visualization parameters, to include imagery (RGB and CIR) and NDVI difference (continuous and discrete color ramps). 

In [11]:
# Create map for visualization
change_map = gm.Map()
change_map.setOptions('SATELLITE')

# Center map to study area
change_map.setCenter(-75.0978, 3.7722, 12)

# Add pre-change and post-change images to map, RGB and CIR
change_map.addLayer(
    peak_green,
    vc.vis_params_rgb,
    'Landsat 8 - RGB - 2017 - Semester A - Peak Green - Pre-Change')

change_map.addLayer(
    post_harvest,
    vc.vis_params_rgb,
    'Landsat 8 - RGB - 2017 - Semester A - Post Harvest - Post-Change')

change_map.addLayer(
    peak_green,
    vc.vis_params_cir,
    'Landsat 8 - CIR - 2017 - Semester A - Peak Green - Pre-Change')

change_map.addLayer(
    post_harvest,
    vc.vis_params_cir,
    'Landsat 8 - CIR - 2017 - Semester A - Post Harvest - Post-Change')

# Add NDVI difference to map, both uncategorized and categorized with SLD
change_map.addLayer(
    ndvi_diff,
    vc.vis_params_ndvi_diff,
    "NDVI Difference - Continuous - 2017 - Semester A - Peak Green to Post-Harvest - Pre to Post-Change")

change_map.addLayer(
    ndvi_diff.sldStyle(vc.vis_params_ndvi_sld),
    {},
    'NDVI Difference - 5 Intervals/Categories - 2017 - Semester A - Peak Green to Post-Harvest - Pre to Post-Change')

# Add classified/extracted rasters (primary and secondary)
change_map.addLayer(
    change_features.get('primary'),
    {'palette': ['green']},
    "Classified Clusters - Raster - Primary Change - 2017 Semester A - Peak Green to Post-Harvest - Pre to Post-Change")

change_map.addLayer(
    change_features.get('secondary'),
    {'palette': ['lightgreen']},
    "Classified Clusters - Raster - Secondary Change - 2017 Semester A - Peak Green to Post-Harvest - Pre to Post-Change")

# Add classified/extracted vectors (from workflow internal)
change_map.addLayer(
    change_primary_vector,
    {'color': 'green'},
    "Classified Clusters - Vector - Primary Change - 2017 Semester A - Peak Green to Post-Harvest - Pre to Post-Change")

change_map.addLayer(
    change_secondary_vector,
    {'color': 'lightgreen'},
    "Classified Clusters - Vector - Secondary Change - 2017 Semester A - Peak Green to Post-Harvest - Pre to Post-Change")

# Add classified/extracted vectors (from workflow GEE Asset export)
change_map.addLayer(
    change_primary_asset,
    {'color': 'green'},
    "Classified Clusters - GEE Asset - Primary Change - 2017 Semester A - Peak Green to Post-Harvest - Pre to Post-Change")

change_map.addLayer(
    change_secondary_asset,
    {'color': 'lightgreen'},
    "Classified Clusters - GEE Asset - Secondary Change - 2017 Semester A - Peak Green to Post-Harvest - Pre to Post-Change")

# Add study area boundary and canals to map
empty = ee.Image().byte()

study_area_boundary_vis = empty.paint(
    featureCollection=study_area_boundary, color=1, width=3)

study_area_canals_vis = empty.paint(
    featureCollection=study_area_canals, color=1, width=3)

change_map.addLayer(
    study_area_boundary_vis,
    {'palette': 'FF0000'},
    'Study Area - Boundary')

change_map.addLayer(
    study_area_canals_vis,
    {'palette': 'blue'},
    'Study Area - Canals')

The workflow displays the interative map that contains all layers used in the analysis, to include:

* Study area vector files;
* Pre and post-change imagery (RGB and CIR); 
* NDVI difference image;
* Classified clusters (rasters); 
* Classified clusters (vectors) from the raster to vector conversion; and,
* Classified clusters (vectors) imported from the Google Earth Engine Assets which the workflow exported during the Data Export step.  

In [12]:
# Display map
change_map

Map(center=[3.7722, -75.0978], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…